In [ ]:
import re
import numpy as np
import pandas as pd
import mne
from scipy import stats
from sklearn.preprocessing import MinMaxScaler, StandardScaler
import matplotlib.pyplot as plt
import seaborn as sns
import time
import multiprocessing as mp
from joblib import Parallel, delayed
from pandarallel import pandarallel
from entropy import *
from itertools import combinations, permutations
from sklearn import preprocessing
from sklearn.model_selection import cross_val_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
import csv
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from statsmodels.stats.outliers_influence import variance_inflation_factor
from statsmodels.tools.tools import add_constant
from sklearn.linear_model import LogisticRegression
from sklearn import tree
from mne_features.feature_extraction import extract_features
from mne_features import *
from scipy import stats
import os
from numba import jit
import math
import scipy.signal as signal
from tqdm import tqdm
import math
from sklearn import feature_selection
import sklearn_relief as relief
from sklearn.linear_model import Lasso
from sklearn.model_selection import KFold
from sklearn.metrics import classification_report,confusion_matrix,cohen_kappa_score
from hmmlearn import hmm
from PyEMD import EMD
from PyEMD import EMD, Visualisation
import statsmodels.tsa.api as smt
from sklearn.feature_selection import SelectFromModel
from sklearn.svm import LinearSVC
from sklearn.model_selection import learning_curve
from sklearn.model_selection import validation_curve
from sklearn import tree
from sklearn.model_selection import GridSearchCV
%matplotlib inline

In [ ]:
tr_path = "C:/Users/vita7/OneDrive/Documents/Github/EEG_SleepStage_Analysis/data/SC/training_nms/"
te_path = "C:/Users/vita7/OneDrive/Documents/Github/EEG_SleepStage_Analysis/data/SC/testing/"

record_regu = r"SC\d*\D\d\-PSG.edf"
annot_regu = r"SC\d*\D\D\-Hypnogram.edf"

channel_types = {
    'EEG Fpz-Cz': 'eeg',
    'EEG Pz-Oz': 'misc',
    'EOG horizontal': 'misc',
    'Resp oro-nasal': 'misc',
    'EMG submental': 'misc',
    'Temp rectal': 'misc',
    'Event marker': 'misc'
}

sleep_stage = {
    'Sleep stage W': 1,
    'Sleep stage 1': 2,
    'Sleep stage 2': 3,
    'Sleep stage 3': 4,
    'Sleep stage 4': 4,
    'Sleep stage R': 5
}

event_id = {
    'Sleep stage W': 1,
    'Sleep stage 1': 2,
    'Sleep stage 2': 3,
    'Sleep stage 3/4': 4,
    'Sleep stage R': 5,
}

funcs_n = [
    'STD', 'Skewness', 'Kurtosis', 'PtP_Amp', 'Zero_crossings', 'Power',
    'Spect_slop', 'Mean_frequency', 'Hjorth_complexity', 'Spect_entropy',
    'Perm_entropy', 'SVD_entropy', 'Sample_entropy', 'AutoCorr'
]

ratio_n = [
    'EEG Fpz-Cz DOR', 'EEG Fpz-Cz TOR', 'EEG Fpz-Cz AOR', 'EEG Fpz-Cz SOR',
    'EEG Fpz-Cz BOR', 'EEG Fpz-Cz TDR', 'EEG Fpz-Cz ADR', 'EEG Fpz-Cz SDR',
    'EEG Fpz-Cz BDR', 'EEG Fpz-Cz ATR', 'EEG Fpz-Cz STR', 'EEG Fpz-Cz BTR',
    'EEG Fpz-Cz SAR', 'EEG Fpz-Cz BAR', 'EEG Fpz-Cz BSR'
]
six_bands_n = [
    'EEG Fpz-Cz', 'EEG Fpz-Cz Delta', 'EEG Fpz-Cz Theta', 'EEG Fpz-Cz Alpha',
    'EEG Fpz-Cz Sigma', 'EEG Fpz-Cz Beta'
]

frbands_n = six_bands_n + ratio_n

ep_duration = 10.
sfreq = 100.
smoothing_factor = 3
l_freq = 0.5
h_freq = 30
chs = 1
funcs = 14

partial_bands = [[0, 5, 7, 8, 20], [1, 5, 20], [6], [7], [0], [16, 19],
                 [2, 6, 18, 19], [6, 19], [1, 4, 9, 13, 14], [20]]